In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(-20, 20, 0.1)
y = np.sin(x)
plt.plot(x, y)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

a = tf.constant(100, name='a')
b = tf.constant(50, name='b')
add_op = a + b

# 変数を宣言してadd_opの結果を代入
v = tf.Variable(0, name='v')
let_op = tf.assign(v, add_op)

# すべての変数を初期化して変数の中身を表示
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(let_op)
print(sess.run(v))

In [16]:
import tensorflow as tf

# 配列を設定(Noneにすると可変長になる)
a = tf.placeholder(tf.int32, [None])
b = tf.constant(2)
x2_op = a * b

# セッションを開始
sess = tf.Session()

# プレースホルダーに値を当てはめて実行する（[1, 2, 3] * 2を実行)
r1 = sess.run(x2_op, feed_dict={ a:[1, 2, 3] })
print(r1)

r2 = sess.run(x2_op, feed_dict={ a:[10, 20, 30] })
print(r2)

[2 4 6]
[20 40 60]


In [17]:
# bmi.csvを作成する

import sys
sys.path.append('/usr/local/lib/python3.5/dist-packages')

import random

# BMIを計算して体型を返す
def calc_bmi(h, w):
    bmi = w / (h/100) ** 2
    if bmi < 18.5: return "thin"
    if bmi < 25.0: return "normal"
    return "fat"

# 出力ファイルの準備
fp = open("bmi.csv","w",encoding="utf-8")
fp.write("height,weight,label\r\n")

# ランダムなデータを生成
cnt = {"thin":0, "normal":0, "fat":0}
for i in range(20000):
    h = random.randint(120,200)
    w = random.randint(35, 80)
    label = calc_bmi(h, w)
    cnt[label] += 1
    fp.write("{0},{1},{2}\r\n".format(h, w, label))
fp.close()


In [61]:
import numpy as np
import pandas as pd
import tensorflow as tf

# ファイル読み込み
csv = pd.read_csv('bmi.csv')

# 適当に正規化
csv['height'] = csv['height'] / 200
csv['weight'] = csv['weight'] / 100

#  ラベルを三次元のクラスで表す
bclass = {'thin': [1,0,0], 'normal': [0,1,0], 'fat': [0,0,1]}
# 三次元の配列が入っている配列にする
csv['label_array'] = csv['label'].apply(lambda x : np.array(bclass[x]))

# テストデータの準備(重さ・高さの配列と、三次元配列の配列を抽出)
test_csv = csv[15000:20000] 
test_pat = test_csv[['weight', 'height']]
test_ans = list(test_csv['label_array'])

# 身長・体重、答えのラベルを格納する変数
x = tf.placeholder(tf.float32, [None, 2])
y_ = tf.placeholder(tf.float32, [None, 3])

# 重みとバイアスの宣言
W = tf.Variable(tf.zeros([2, 3]))
b = tf.Variable(tf.zeros([3]))

# 予測結果を定義(matmulはnumpyのdotと同じ) ソフトマックスで出力した結果
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 訓練（損失関数を定義して、勾配降下法で損失関数を最小にする）
#　答えのラベルと予測結果を使用する
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(cross_entropy)

# 精度を求める（答えと予測結果が正しいか）
predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

# セッションの開始
sess = tf.Session()
tw = tf.summary.FileWriter('log_dir', graph=sess.graph)
sess.run(tf.global_variables_initializer())

# 学習
for step in range(3500):
    # 100件ずつミニバッチで学習させる
    i = (step * 100) % 14000
    rows = csv[1 + i : 1 + i + 100]
    
    # 学習データとその答えの用意
    x_pat = rows[['weight', 'height']]
    y_ans = list(rows['label_array'])
    fd = {x: x_pat, y_: y_ans}

    sess.run(train, feed_dict=fd)

    # 500件ごとに損失関数と精度を表示する
    if step % 500 == 0:
        # 学習データを使って誤差を求める
        cre = sess.run(cross_entropy, feed_dict=fd)
        # テストデータを使って精度を求める
        acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
        print('step=', step, 'cre=', cre, 'acc=', acc)

acc = sess.run(accuracy, feed_dict={x: test_pat, y_: test_ans})
print('最終的な正解率=', acc)
        

step= 0 cre= 108.91 acc= 0.3126
step= 500 cre= 46.6868 acc= 0.809
step= 1000 cre= 40.8409 acc= 0.8908
step= 1500 cre= 40.1824 acc= 0.943
step= 2000 cre= 35.9449 acc= 0.9188
step= 2500 cre= 32.0373 acc= 0.9612
step= 3000 cre= 32.5656 acc= 0.968
最終的な正解率= 0.966
